In [1]:
import numpy as np
import pandas as pd
import math
from datetime import datetime
import CMAPSAuxFunctions

from data_handler_VALVE import ValveDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LeakyReLU
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


<h1> Create Data Handler </h1>

In [2]:
K.clear_session()

features = ['id', 'timestamp', 'externalControllerOutput',
            'pressureValveInlet', 'pressureValveOutlet', 'mediumTemperature',
            'rodDisplacement', 'disturbedMediumFlow', 'selectedFault',
            'faultType', 'faultIntensity']
selected_indices = np.array([3,4,5,6,7,8])
selected_features = list(features[i] for i in selected_indices-1)

window_size = 30
window_stride = 20

# min = 2018-02-14 18:59:20
# max = 2018-08-19 18:28:20
time_start = "2018-02-14 18:59:20"
time_end = "2018-04-19 18:28:20"

# Either anomaly, classification or regression
#problem = 'anomaly'
problem = 'classification'
#problem = 'regression'

vHandler = ValveDataHandler(time_start, time_end, selected_features = selected_features,
                            sequence_length = window_size, sequence_stride = window_stride,
                            problem = problem)

init


In [3]:
print(selected_features)

['externalControllerOutput', 'pressureValveInlet', 'pressureValveOutlet', 'mediumTemperature', 'rodDisplacement', 'disturbedMediumFlow']


<h1> Keras Model </h1>

In [4]:
#np.set_printoptions(threshold=np.nan)

K.clear_session()
lambda_regularization = 0.20

def create_ANN_model(input_shape, problem):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(8, input_dim = input_shape, activation = 'relu'))
    model.add(LeakyReLU(alpha = 0.001))
    model.add(Dense(15, activation = 'relu'))
    model.add(LeakyReLU(alpha = 0.001))
    model.add(Dense(1, activation = 'sigmoid'))
    
#     if (problem == 'classification' or problem == 'anomaly'):
#         model.add(Dense(1, activation = 'sigmoid', name = 'out'))
#     elif (problem == 'regression'):
#         model.add(Dense(1, activation = 'linear', name = 'out'))
        
    #model.add(Dense(1, activation = 'softmax', name = 'out'))
    
    return model

<h1>Tunable Model </h1>

In [5]:
# Default: Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
# Default: RMSprop(lr=0.001, rho=0.9, epsilon=1e-6)
# Default: SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

def get_compiled_model(shape, problem):
    
    K.clear_session()
    
    if (problem == 'classification' or problem == 'anomaly'):
        # Parameters for the model
        learning_rate = 0.01
        decay_rate = 0
        
        optimizer = SGD(lr = learning_rate, momentum=0.0, decay = decay_rate, nesterov=False)
        loss_function = 'binary_crossentropy'
        metrics = ['binary_accuracy']
        
    elif (problem == 'regression'):
        # Parameters for the model
        optimizer = Adam(lr = 0.1, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = False)
        loss_function = 'mean_squared_error'
        metrics = ['mse']
        
    model = None
    
    # Create and compile the model
    model = create_ANN_model(shape, problem)
    model.compile(optimizer = optimizer, loss = loss_function, metrics = metrics)
    
    return model

In [6]:
scaler = None
num_features = len(selected_features)
input_shape = num_features * window_size
# scaler = MinMaxScaler(feature_range = (0, 1))
scaler = StandardScaler()

model = get_compiled_model(input_shape, problem)
tModel = SequenceTunableModelRegression('ANN_Model', model, lib_type = 'keras', data_handler = vHandler, data_scaler = scaler, batch_size = 16)

<h1> Pulling Data from CSV </h1>

In [7]:
vHandler._df = pd.read_csv('valve_dataset.csv', sep = ',')

vHandler._df['status'] = vHandler._df['selectedFault'].apply(lambda valve: 0 if valve == 20 else 1)

vHandler._X = vHandler._df[selected_features].values
vHandler._y = vHandler._df['status'].values

<h1> Loading Data from MySQL Database </h1>

In [8]:
#vHandler.connect_to_db("remoteAdmin","remoteAdmin","169.236.181.40:3306","damadics")

In [9]:
# Extract data from database
#vHandler.extract_data_from_db()
# vHandler.extract_data_from_db()

In [10]:
print(vHandler._X)
print(vHandler._y)

[[5.00000e-01 8.76840e-01 6.50832e-01 2.14553e-01 1.59000e-03 1.00000e+00]
 [3.66043e-01 8.50576e-01 6.43717e-01 2.15373e-01 3.69939e-01 1.00000e+00]
 [7.32444e-01 8.49537e-01 6.47472e-01 2.14781e-01 7.32446e-01 2.22181e-01]
 ...
 [7.32444e-01 8.47251e-01 6.43362e-01 2.14299e-01 1.00000e+00 0.00000e+00]
 [2.57854e-01 8.49342e-01 6.43664e-01 2.15743e-01 1.00000e+00 3.34000e-04]
 [6.59356e-01 8.49701e-01 6.43186e-01 2.17503e-01 1.00000e+00 0.00000e+00]]
[0 0 0 ... 1 1 1]


In [ ]:
tModel.load_data(unroll = True, verbose = 0, cross_validation_ratio = 0.45, test_ratio = 0.05)

In [ ]:
vHandler.print_data(True)

<h1> Train Model and Test Tunable Model Functionalities </h1>

In [ ]:
# tModel.epochs = 10
# tModel.train_model(verbose = 1)

In [ ]:
# # Predicting the test set
# tModel.predict_model(True)

In [ ]:
K.clear_session()
model = None

model = Sequential()
model.add(Dense(8, input_dim = input_shape, activation = 'elu'))
model.add(LeakyReLU(alpha = 0.001))
model.add(Dense(10, activation = 'elu'))
model.add(LeakyReLU(alpha = 0.001))
model.add(Dense(1, activation = 'sigmoid'))

#opt = 'adam'
#opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
opt = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

callbacks = None
#callbacks = [EarlyStopping(monitor='val_binary_accuracy', patience = 2)]

model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['binary_accuracy'])
model.summary()

In [ ]:
batch_size = 32
epochs = 50

full_history = []

history = model.fit(tModel._X_train, tModel._y_train, batch_size = batch_size,
          validation_data = (tModel._X_crossVal, tModel._y_crossVal),
          callbacks = callbacks,
          epochs = epochs, shuffle = True)

full_history.append(history)

#model.fit(tModel._X_crossVal, tModel._y_crossVal, validation_data = (tModel._X_test, tModel._y_test), epochs = 20)

In [ ]:
predictions = model.predict(tModel._X_crossVal)
#print(np.ravel(predictions))
print(np.ravel(np.round(predictions)))

In [ ]:
print(np.ravel(tModel._y_crossVal))

In [ ]:
middle_value = 0.20

new_predictions = None
new_predictions = []

for i in range(len(predictions)):
    if (predictions[i] <= middle_value):
        new_predictions.append(0.)
    else:
        new_predictions.append(1.)

In [ ]:
print(np.ravel(tModel._y_crossVal))
print(np.ravel(new_predictions))

In [ ]:
scores = model.evaluate(tModel._X_test, tModel._y_test)
print('''Loss of test set: {}
Accuracy of test set: {}%'''
      .format(scores[0], round(scores[1] * 100, 6)))

In [ ]:
num_correct = 0
j = 0

for i in predictions:
    if tModel._y_crossVal[j] == np.round(i):
        num_correct += 1
    j += 1

print("Number of correct predictions: {} out of {}".format(num_correct, len(tModel._y_crossVal)))

In [ ]:
del model